In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

np.random.seed(42)
random.seed(42)

In [2]:
data = pd.read_csv('Project_1_dataset_01_01_2022.csv')


In [3]:
from sklearn.pipeline import Pipeline
from tempfile import mkdtemp
from shutil import rmtree
from joblib import Memory
from nltk.stem import PorterStemmer
import re
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, word_tokenize
from string import punctuation
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV

wnl = nltk.wordnet.WordNetLemmatizer()

def penn2morphy(penntag): 
    morphy_tag = {'NN':'n', 'JJ':'a',
                  'VB':'v', 'RB':'r'}
    try:
        return morphy_tag[penntag[:2]]
    except:
        return 'n' 

def lemmatize_sent(list_word):
    # Text input is string, returns array of lowercased strings(words).
    return [wnl.lemmatize(word.lower(), pos=penn2morphy(tag)) 
            for word, tag in pos_tag(list_word)]

analyzer = CountVectorizer().build_analyzer()

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

def stem_rmv_punc(doc):
    return (word for word in lemmatize_sent(analyzer(doc)) if not word.isdigit())

def with_lemmatization(doc):
    sent = lemmatize_sent(doc) 
    sent = [i for i in sent if i not in punctuation] 
    sent = [i for i in sent if not i.isdigit()] 
    return sent

ps = nltk.stem.PorterStemmer()
def with_stemming(doc):
    sent = ps.stem(doc)
    sent = [i for i in sent if i not in punctuation] 
    sent = [i for i in sent if not i.isdigit()] 
    return sent

def without_lemmatization(doc):
    sent = nltk.word_tokenize(doc)
    sent = [i for i in sent if i not in punctuation] 
    sent = [i for i in sent if not i.isdigit()] 
    return sent

def clean(text):
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    texter = re.sub(r"<br />", " ", text)
    texter = re.sub(r"&quot;", "\"",texter)
    texter = re.sub('&#39;', "\"", texter)
    texter = re.sub('\n', " ", texter)
    texter = re.sub(' u '," you ", texter)
    texter = re.sub('`',"", texter)
    texter = re.sub(' +', ' ', texter)
    texter = re.sub(r"(!)\1+", r"!", texter)
    texter = re.sub(r"(\?)\1+", r"?", texter)
    texter = re.sub('&amp;', 'and', texter)
    texter = re.sub('\r', ' ',texter)
    clean = re.compile('<.*?>')
    texter = texter.encode('ascii', 'ignore').decode('ascii')
    texter = re.sub(clean, '', texter)
    if texter == "":
        texter = ""
    return texter

def create_targetlabel(train, test):
    
    train["root_label"] = train["root_label"].astype('category')
    train["target"] = train["root_label"].cat.codes
    train["root_label"] = train["root_label"].astype('object')

    test["root_label"] = test["root_label"].astype('category')
    test["target"] = test["root_label"].cat.codes
    test["root_label"] = test["root_label"].astype('object')
    
cachedir = mkdtemp()
memory = Memory(location=cachedir, verbose=10)

    
svd_50 = TruncatedSVD(n_components=50, random_state=42)
svd_500 = TruncatedSVD(n_components=500, random_state=42)
SVM_best_Gamma = SVC(kernel='linear',C=500,random_state=42)
clf_best_L1 = LogisticRegression(penalty='l1', C=75, random_state=42, solver='liblinear')
clf_best_L2 = LogisticRegression(penalty='l2', C=1000, random_state=42, solver='liblinear')

data_withcleaning = data.copy()
data_withcleaning['full_text'] = data_withcleaning['full_text'].map(lambda text: clean(text))
train_withcleaning, test_withcleaning = train_test_split(data_withcleaning[["full_text","root_label"]], test_size = 0.2, random_state = 42)
train_withoutcleaning, test_withoutcleaning = train_test_split(data[["full_text","root_label"]], test_size = 0.2, random_state = 42)
create_targetlabel(train_withcleaning, test_withcleaning)
create_targetlabel(train_withoutcleaning, test_withoutcleaning)

print("cleaning train shape:", train_withcleaning.shape)
print("cleaning test shape:", test_withcleaning.shape)
print("without cleaning train shape:", train_withoutcleaning.shape)
print("without cleaning test shape:", test_withoutcleaning.shape)


pipeline = Pipeline([('vect',TfidfVectorizer(stop_words='english')),
                     ('tfidf', TfidfTransformer()),
                     ('reduce_dim',None),
                     ('clf',None)], memory=memory)

param_grid = {
    'vect__min_df': (3,5),
    'vect__analyzer': (with_lemmatization, with_stemming, without_lemmatization),
    'reduce_dim':(TruncatedSVD(n_components=5,random_state=42), svd_50, svd_500, NMF(n_components=5,init='random', random_state=42), NMF(n_components=50, init='random', random_state=42), NMF(n_components=500,init='random', random_state=42)),
    'clf':(SVM_best_Gamma, clf_best_L1, clf_best_L2, GaussianNB())
    } # parameter grid for the pipeline to find the best combination


grid1 = GridSearchCV(pipeline,cv=5,param_grid=param_grid,scoring='accuracy')
grid2 = GridSearchCV(pipeline,cv=5,param_grid=param_grid,scoring='accuracy')

grid1.fit(train_withcleaning['full_text'], train_withcleaning['target'])
grid2.fit(train_withoutcleaning['full_text'], train_withoutcleaning['target'])
rmtree(cachedir)



cleaning train shape: (1657, 3)
cleaning test shape: (415, 3)
without cleaning train shape: (1657, 3)
without cleaning test shape: (415, 3)
________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(TfidfVectorizer(analyzer=<function with_lemmatization at 0x00000238B0877670>,
                min_df=3, stop_words='english'), 
1782    "Pages of history" features excerpts from The ...
1909    Weather Rain Saturday Will Help With Symptoms ...
1920    Will the serious drought conditions afflicting...
1005    In addition to the new desktop interface, Micr...
1538    A major storm system has brought heavy rain an...
                              ...                        
1638    The redesigned Ford Ranger made its debut on W...
1095    (Newser)  A family that died on a hike in Cali...
1130    --News Direct-- Pano AI , the leader in wildfi...
1294    EAST TAWAS, Mich.  The public is invited to a 

______________________________________________fit_transform_one - 171.6s, 2.9min
________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(TfidfTransformer(), <1326x27 sparse matrix of type '<class 'numpy.float64'>'
	with 34328 stored elements in Compressed Sparse Row format>, 
1431    0
1567    0
1044    0
175     1
1610    0
       ..
1638    0
1095    0
1130    0
1294    0
860     1
Name: target, Length: 1326, dtype: int8, 
None, message_clsname='Pipeline', message=None)
________________________________________________fit_transform_one - 0.0s, 0.0min
________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(TruncatedSVD(n_components=5, random_state=42), <1326x27 sparse matrix of type '<class 'numpy.float64'>'
	with 34328 stored elements in Compressed Sparse Row format>, 
1431    0
1567    0

______________________________________________fit_transform_one - 166.7s, 2.8min
[Memory]166.7s, 2.8min  : Loading _fit_transform_one from C:\Users\ericz\AppData\Local\Temp\tmpfcc3jzg_\joblib\sklearn\pipeline\_fit_transform_one\de08d987bba6b4b271e7742180545796
___________________________________fit_transform_one cache loaded - 0.0s, 0.0min
[Memory]166.7s, 2.8min  : Loading _fit_transform_one from C:\Users\ericz\AppData\Local\Temp\tmpfcc3jzg_\joblib\sklearn\pipeline\_fit_transform_one\76cb487047c779f85dece80bdbad5dc4
___________________________________fit_transform_one cache loaded - 0.0s, 0.0min
________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(TfidfVectorizer(analyzer=<function with_lemmatization at 0x00000238B0877670>,
                min_df=5, stop_words='english'), 
1431    More than 1 million Washingtonians participate...
1567    A 4.7-magnitude earthquake rattled northwest S.

________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(TfidfVectorizer(analyzer=<function with_stemming at 0x00000238B0877700>,
                min_df=3, stop_words='english'), 
1431    More than 1 million Washingtonians participate...
1567    A 4.7-magnitude earthquake rattled northwest S...
1044    More campgrounds are coming to the Auburn Stat...
175     Retro gamers will be able to relive all the jo...
1610    SAN SIMEON, Calif. (KGTV)  An earthquake with ...
                              ...                        
1638    The redesigned Ford Ranger made its debut on W...
1095    (Newser)  A family that died on a hike in Cali...
1130    --News Direct-- Pano AI , the leader in wildfi...
1294    EAST TAWAS, Mich.  The public is invited to a ...
860     If theres a central theme to Edgar Wrights new...
Name: full_text, Length: 1326, dtype: object, 
1431    0
1567    0
1044    0
175   

________________________________________________fit_transform_one - 1.0s, 0.0min
[Memory]1.0s, 0.0min    : Loading _fit_transform_one from C:\Users\ericz\AppData\Local\Temp\tmpfcc3jzg_\joblib\sklearn\pipeline\_fit_transform_one\1af414203ac777bff3e7471f58ebd1b8
___________________________________fit_transform_one cache loaded - 0.0s, 0.0min
[Memory]1.0s, 0.0min    : Loading _fit_transform_one from C:\Users\ericz\AppData\Local\Temp\tmpfcc3jzg_\joblib\sklearn\pipeline\_fit_transform_one\c97254ec1a33cbed936742770b85dbd2
___________________________________fit_transform_one cache loaded - 0.0s, 0.0min
________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(TfidfVectorizer(analyzer=<function with_stemming at 0x00000238B0877700>,
                min_df=5, stop_words='english'), 
1431    More than 1 million Washingtonians participate...
1567    A 4.7-magnitude earthquake rattled northwest S...
10

...

C:\Users\ericz\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1076: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


[Memory]0.0s, 0.0min    : Loading _fit_transform_one from C:\Users\ericz\AppData\Local\Temp\tmpfcc3jzg_\joblib\sklearn\pipeline\_fit_transform_one\335ea5ce047b530b7bd6be31b9c33f32
___________________________________fit_transform_one cache loaded - 0.0s, 0.0min
[Memory]0.0s, 0.0min    : Loading _fit_transform_one from C:\Users\ericz\AppData\Local\Temp\tmpfcc3jzg_\joblib\sklearn\pipeline\_fit_transform_one\947b7d7ad9a1584df41d103d6b353d04
___________________________________fit_transform_one cache loaded - 0.0s, 0.0min
[Memory]0.0s, 0.0min    : Loading _fit_transform_one from C:\Users\ericz\AppData\Local\Temp\tmpfcc3jzg_\joblib\sklearn\pipeline\_fit_transform_one\31047cdfed83265f79711872804b65eb
___________________________________fit_transform_one cache loaded - 0.0s, 0.0min


C:\Users\ericz\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1076: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


[Memory]0.0s, 0.0min    : Loading _fit_transform_one from C:\Users\ericz\AppData\Local\Temp\tmpfcc3jzg_\joblib\sklearn\pipeline\_fit_transform_one\98d0ed80c0d748a7e9ad28370a5d2121
___________________________________fit_transform_one cache loaded - 0.0s, 0.0min
[Memory]0.1s, 0.0min    : Loading _fit_transform_one from C:\Users\ericz\AppData\Local\Temp\tmpfcc3jzg_\joblib\sklearn\pipeline\_fit_transform_one\d56f56f09814459c2f5e7aa67cde1b4f
___________________________________fit_transform_one cache loaded - 0.0s, 0.0min
[Memory]0.1s, 0.0min    : Loading _fit_transform_one from C:\Users\ericz\AppData\Local\Temp\tmpfcc3jzg_\joblib\sklearn\pipeline\_fit_transform_one\83af55de84b295a7195f8ba812a37819
___________________________________fit_transform_one cache loaded - 0.0s, 0.0min
[Memory]0.0s, 0.0min    : Loading _fit_transform_one from C:\Users\ericz\AppData\Local\Temp\tmpfcc3jzg_\joblib\sklearn\pipeline\_fit_transform_one\ac1543c3c296b27daf0d41c3aee5e563
___________________________________fi

________________________________________________fit_transform_one - 4.6s, 0.1min
________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(TfidfTransformer(), <1657x10244 sparse matrix of type '<class 'numpy.float64'>'
	with 330440 stored elements in Compressed Sparse Row format>, 
1431    0
1567    0
1044    0
175     1
1610    0
       ..
1638    0
1095    0
1130    0
1294    0
860     1
Name: target, Length: 1657, dtype: int8, 
None, message_clsname='Pipeline', message=None)
________________________________________________fit_transform_one - 0.0s, 0.0min
________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(TruncatedSVD(n_components=500, random_state=42), <1657x10244 sparse matrix of type '<class 'numpy.float64'>'
	with 330440 stored elements in Compressed Sparse Row format>, 
1431    0

In [9]:
your_result1 = pd.DataFrame(grid1.cv_results_)
your_result1['clean&not_clean'] = 'cleaned'
your_result2 = pd.DataFrame(grid2.cv_results_)
your_result2['clean&not_clean'] = 'not_cleaned'
grid_Q8 = pd.concat([your_result1, your_result2], ignore_index = True)
grid_sorted = grid_Q8.sort_values(by = 'rank_test_score')
pd.options.display.max_columns = None
pd.options.display.width=None
grid_top5 = grid_sorted.head(5)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(grid_top5)
    


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf,param_reduce_dim,param_vect__analyzer,param_vect__min_df,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,clean&not_clean
53,0.326634,0.029494,0.929170,0.041426,"LogisticRegression(C=75, penalty='l1', random_...","TruncatedSVD(n_components=500, random_state=42)",<function without_lemmatization at 0x00000238B...,5,"{'clf': LogisticRegression(C=75, penalty='l1',...",0.978916,0.981928,0.963746,0.969789,0.972810,0.973438,0.006476,1,cleaned
197,0.327201,0.034574,0.843600,0.026263,"LogisticRegression(C=75, penalty='l1', random_...","TruncatedSVD(n_components=500, random_state=42)",<function without_lemmatization at 0x00000238B...,5,"{'clf': LogisticRegression(C=75, penalty='l1',...",0.975904,0.981928,0.960725,0.960725,0.969789,0.969814,0.008355,1,not_cleaned
52,0.391805,0.048135,0.902396,0.030528,"LogisticRegression(C=75, penalty='l1', random_...","TruncatedSVD(n_components=500, random_state=42)",<function without_lemmatization at 0x00000238B...,3,"{'clf': LogisticRegression(C=75, penalty='l1',...",0.975904,0.978916,0.963746,0.966767,0.963746,0.969816,0.006369,2,cleaned
196,0.353034,0.029424,0.870808,0.037856,"LogisticRegression(C=75, penalty='l1', random_...","TruncatedSVD(n_components=500, random_state=42)",<function without_lemmatization at 0x00000238B...,3,"{'clf': LogisticRegression(C=75, penalty='l1',...",0.969880,0.975904,0.963746,0.963746,0.969789,0.968613,0.004550,2,not_cleaned
89,0.355823,0.027560,0.925797,0.029698,"LogisticRegression(C=1000, random_state=42, so...","TruncatedSVD(n_components=500, random_state=42)",<function without_lemmatization at 0x00000238B...,5,"{'clf': LogisticRegression(C=1000, random_stat...",0.975904,0.984940,0.960725,0.960725,0.963746,0.969208,0.009655,3,cleaned


This is the exact consecutive process which is supposed to complete in the same file. 
Since I ran this single Question 8 for over 20 hours, there is no chance to combine files together to run again, or I may pass the ddl.

For any chance if you do not see my testing performances, please refer to page 2 of this pdf file.